# try doc2vec

In [ ]:
# model = Doc2Vec(contentcutlist, size=100, window=8, min_count=5, workers=4)
import pandas as pd
import numpy as np
import jieba
#改為中文繁體字典,預設是簡體，可到jieba的GitHub下載
jieba.set_dictionary(r'.\data\jieba_dict.txt.big') 
from wordcloud import WordCloud
import random
import json

In [ ]:
with open(r".\data\bigtable_1.3.json") as f:
    BigAnalyzeTable=json.load(f)


In [ ]:
def StopWdFilter(user_input, stop_words):
    """Sanitize using intersection and list.remove()"""
    # Downsides:
    #   - Looping over list while removing from it?
    #     http://stackoverflow.com/questions/1207406/remove-items-from-a-list-while-iterating-in-python
    stop_words = set()
    with open('./dict/stopwords.txt', 'r', encoding='utf-8') as sw:
        for line in sw:
            stop_words.add(line.strip('\n'))
    for sw in stop_words.intersection(user_input):
        while sw in user_input:
            user_input.remove(sw)

    return user_input

In [ ]:
def CutWithFilter(CorpusList):
    result=[]
    for y in [jieba.cut(x) for x in CorpusList]:
        content = StopWdFilter([ _ for _ in y], stopwords)
        result.append(content)
    return result
#所以我選擇所過濾的是跟「文意」比較無關的符號
stopwords = ['〔','〕','／','（','）','「','」','『','』','\n','nan']
#像下面這種比較是傳統文字探勘在過濾字詞可能過濾的一些字詞
#stopwords = ['〔','〕','／','（','）','「','」','『','』','的','就是','這個','可以','一定'\
#             ,'一個','nan','沒有','我們','不是','不用','不是','自己','只要','大家','覺得','因為'\
#             ,'對於','起來','如果','每個','很多','一樣','時候','加上','使用','只是']

In [ ]:
#只留下店名跟文章做分類分析
FoodArticle=[{'name':dien["name"],'content':"".join(dien['contentcut'].split(" "))} for dien in BigAnalyzeTable]

In [ ]:
FoodArticle[0]

In [ ]:
Food_df=pd.DataFrame(FoodArticle)

In [ ]:
Food_df

In [ ]:
Food = Food_df.name+ " " + Food_df.content

In [ ]:
Food

In [ ]:
Food_List = CutWithFilter(Food)

In [ ]:
Food_List[0]

# 6. 把gensim, Doc2Vec的套件匯入

In [ ]:
import gensim
Lsent = gensim.models.doc2vec.LabeledSentence
def labelizeNews(food, label_type):
    labelized = []
    #enumertate include index
    for i, v in enumerate(food):
        label = '%s_%s' %(label_type,i)
        labelized.append(Lsent(v, [label]))
    return labelized


# for i,v in enumerate()
# i為流水號，1  2  3
# v為每篇文章的，第1個字全部  第2個字全部  第3個字全部
# label_type為type標籤
# labelized = []
# for i,v in enumerate(Food_List):
#     label = '%s_%s' %('Food',i)
#     labelized.append(Lsent(v, [label]))

# 將第一篇的文字全部上Food_1的標籤
# 將第二篇的文字全部上Food_2的標籤
# 以此類推

# 7. 將每個的文章給一個標籤，方便分析完對應

In [ ]:
Train_Food = labelizeNews(Food_List,'Food')

In [ ]:
print(type(Train_Food))
print(type(Train_Food[1]))
Train_Food[0]#為上完標籤的gensimlabel＿

# 設定Doc2Vec參數 (使用seed or save model 來排除訓練的隨機成分，如果需要每次的向量結果相同 每一個Corpus產出400維的向量，預設使用3個核心做運算

In [ ]:
size = 400   
model_dm = gensim.models.Doc2Vec(min_count=1, window=10, size=size, sample=1e-3, negative=5, workers=3)
model_dbow = gensim.models.Doc2Vec(min_count=1, window=10, size=size, sample=1e-3, negative=5,dm=0, workers=3)

# 由有順序的句子建立model詞彙(讓model讀懂用的) 輸入的資料必須是((文章的有順序斷詞list)的list)

In [ ]:
model_dm.build_vocab(Train_Food)
model_dbow.build_vocab(Train_Food)

# 訓練十次，每次隨機交換文章順序 (由於是隨機, 要每次結果完全相同也可以固定訓練次序)

In [ ]:
for epoch in range(10):
    
    #總共幾篇文章(假設225) 對1~225做隨機排序的array
    perm = np.random.permutation(len(Train_Food))
    
    #將所有的文章隨機順序丟進去訓練 並告知model 共有225個example  5種時?
    model_dm.train([Train_Food[x] for x in perm],total_examples=model_dm.corpus_count,epochs=model_dm.iter)
    model_dbow.train([Train_Food[x] for x in perm],total_examples=model_dbow.corpus_count,epochs=model_dbow.iter)

In [ ]:
perm

# 可以把模型存起來，下次直接取回向量

In [ ]:
model_dm.save('./data/doc2verData/model_dm')
model_dbow.save('./data/doc2verData/model_dbow')

# 取出文章向量

In [ ]:
def getVecs(model, corpus, size):
    #取出每一個tag(Eat_1.Eat_2...Ser_1.Ser_2....)的向量(docvecs)，並將向量長度全都變成400size
    vecs = [model.docvecs[z.tags[0]].reshape((1, size)) for z in corpus]
    
    return np.concatenate(vecs)

In [ ]:
np.concatenate(model_dm.docvecs['Food_3'].reshape((1, 400)))

# 取出兩個模型 dm, dbow 取得其兩個模型的向量

In [ ]:
Train_Food_vecs_dm = getVecs(model_dm, Train_Food, size)
Train_Food_vecs_dbow = getVecs(model_dbow, Train_Food, size)
#將兩種模型的vec上下疊加
Food_train_vecs = np.hstack((Train_Food_vecs_dm, Train_Food_vecs_dbow))

# 載入scikit learn 套件 kmeans; 並將Doc2Vec的向量做分群 將所有食記嘗試分50群，並人工對群體做觀察

In [ ]:
from sklearn.cluster import KMeans
Ngroup=50
#固定random state 隨機初始值 fixed the seed
Food_kmeans = KMeans(n_clusters=Ngroup, random_state=0).fit(Food_train_vecs)

In [ ]:
#清一下記憶體
import gc
gc.collect()

# 將分群結果塞回DataFrame

In [ ]:
# Food_df
# Food_List

#移除,的方法
Food_df['name']=Food_df['name'].apply(lambda y:y.replace(',',""))
Food_df['name']=Food_df['name'].apply(lambda y:y.replace('\n',"")) 

In [ ]:
Food_df['Cluster'] = Food_kmeans.labels_

In [ ]:
Food_df.to_excel('./data/doc2verData/Food_'+str(Ngroup)+'gp.xlsx')
Food_df.to_csv('./data/doc2verData/Food_'+str(Ngroup)+'gp.csv')

In [ ]:
gc.collect()

# 試畫群組文字雲

In [ ]:
# WC_Str=''
# for i, x in enumerate(Food_kmeans.labels_):
#     if x == 2:
#         WC_Str += ' '.join(Food_List[i])

# ww = WordCloud(max_font_size=100, relative_scaling=1,max_words=40,width=800, height=600,background_color='white',random_state=0,\
#                stopwords=["臺南","菜色","味道","真的","店家","這道","感覺","比較","用餐","搭配",
#          "料理","空間","一個","地方","台南市","台南","口味","店員",'一起','一段',
#          '部分','餐點','地方','覺得','廁所','二樓',"看到","市場","地址","三段","一段",
#          '兩段','中西區','海安','一盒','會館','座位','選擇','下午','提供','制式','使用',
#          '北區','營業時間','電話','美食','一份','朋友','東西','一下','來到','一些','二段',
#          '有點','應該','老闆','推薦','這家','一直','菜單','想法','平路','府連',
#          '喜歡','東區','整個','時間','店內','國華','台北','聽說','升級','套餐',
#          '餐廳','相當','店內','裡面','一點','一口','知道','已經','這是','客人'])\
#                .generate(WC_Str)
# ww.font_path = 'C:\Windows\Fonts\msjhbd' #微軟正黑體的路徑；字體可以自己設定，預設是英文的遇到中文會讀不出來
# ww.to_image()
# #ww.to_file('./pic/50gp_5.jpg') 可輸出圖檔

In [ ]:
with open('./data/doc2verData/Food_'+str(Ngroup)+'gp.csv') as f:
    x=f.readlines()

# 發現有些群體過小 將它們合併為一個，並將超少評論或無評的文章群做標記

In [ ]:
kfood=[dien.replace('\n',"").split(',')[2:] for dien in x[1:]]

In [ ]:
groups={}
for dien in kfood:
    if dien[1] not in groups:
        groups[dien[1]]=set()
    groups[dien[1]].add(dien[0])

In [ ]:
groups.keys()

In [ ]:
groups['50']=set()
willdel=set()
for group in groups:
    if len(groups[group])<40:
        groups['50'].update(groups[group])
        willdel.add(group)

In [ ]:
for group in willdel:
    del groups[group]

In [ ]:
g=0
newgroups={}
for i in groups:
    if i =="0":
        newgroups['無評']=list(groups[i])
    elif i =="50":
        newgroups['無分類']=list(groups[i])
    else:
        g+=1
        newgroups[str(g)]=list(groups[i])

In [ ]:
print(len(newgroups))
print([len(newgroups[i]) for i in newgroups])
#為了存成json 轉格式
# x=[dict({i:newgroups[i]}) for i in newgroups]

In [ ]:
import json
with open('./data/doc2verData/Food_kmeans.json','w') as f:
    json.dump(newgroups,f)